In [ ]:
# @title Baixar Bibliotecas e Definir Funções (Rodar Apenas Uma Vez)
# @markdown <b>Esta célula configura o ambiente com todas as bibliotecas e funções necessárias.<br>Execute apenas uma vez no início.</b>

import subprocess
import time
from IPython.display import display
import ipywidgets as widgets
import shutil
import os
import importlib.util
from tqdm.notebook import tqdm

# Lista de etapas (nome, comando, checagem de instalação)
etapas = [
    ("paddlepaddle-gpu", "python3 -m pip install paddlepaddle-gpu",
     lambda: importlib.util.find_spec("paddle")),

    ("paddleocr", "pip install 'paddleocr>=2.0.1'",
     lambda: importlib.util.find_spec("paddleocr")),

    ("openpyxl", "pip install openpyxl",
     lambda: importlib.util.find_spec("openpyxl")),

    ("pacotes: jpype1, aspose-cells, layoutparser, pandas, pdf2image",
     "pip install jpype1 aspose-cells layoutparser pandas pdf2image",
     lambda: all(importlib.util.find_spec(pkg) for pkg in ["jpype", "pandas", "pdf2image", "layoutparser"])),

    ("protobuf==3.20.0", "pip install protobuf==3.20.0",
     lambda: importlib.util.find_spec("google")),

    ("poppler-utils", "apt-get install poppler-utils -y",
     lambda: shutil.which("pdftoppm") is not None),

    ("pikepdf", "pip install -q pikepdf",
     lambda: importlib.util.find_spec("pikepdf")),

    ("ipywidgets", "pip install -q ipywidgets",
     lambda: importlib.util.find_spec("ipywidgets")),

    ("Clonar PaddleOCR", "git clone https://github.com/PaddlePaddle/PaddleOCR.git",
     lambda: os.path.isdir("PaddleOCR")),

    ("Baixar layoutparser .whl", "wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl",
     lambda: os.path.exists("layoutparser-0.0.0-py3-none-any.whl")),

    ("Instalar layoutparser .whl", "pip install -U layoutparser-0.0.0-py3-none-any.whl",
     lambda: importlib.util.find_spec("layoutparser")),
]

print("⚙️ Verificando e instalando bibliotecas necessárias...\n")

# Barra de progresso geral
barra = tqdm(etapas, desc="Instalando", unit="etapa", leave=False)

for nome, comando, checagem in barra:
    barra.set_description(f"🔍 Verificando {nome}")

    if checagem():
        time.sleep(0.1)  # simula verificação
        barra.write(f"✅ {nome} já instalado.")
        continue

    barra.set_description(f"🔧 Instalando {nome}")
    try:
        subetapas = tqdm(range(10), desc=f"⏳ Instalando {nome}", leave=False)
        for _ in subetapas:
            time.sleep(0.05)  # simula progresso suave

        subprocess.run(comando, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        barra.write(f"✅ {nome} instalado com sucesso.")
    except subprocess.CalledProcessError:
        barra.write(f"❌ Erro ao instalar {nome}")

print("\n✅ Ambiente pronto! Todas as dependências estão no lugar.")

# =================== IMPORTS ===================
import os
import logging
import unicodedata
import re
import shutil
from openpyxl.styles import Font
from decimal import Decimal, InvalidOperation
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from PIL import Image
from pdf2image import convert_from_path
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import pikepdf
from paddleocr import PaddleOCR
import layoutparser as lp

# Silenciar logs do OCR
logging.getLogger("ppocr").setLevel(logging.ERROR)

# Inicializa OCR
ocr = PaddleOCR(use_angle_cls=True, lang='pt')

print("\n🚀 Bibliotecas carregadas com sucesso!")




mapeamento_tipo_cobranca = {
#===================================================================ENCARGO================================================================================================================
        "Energia elet. adquirida 3": "Encargo",
        "energia elet adquirida 3": "Encargo",
        "energiaelet.adquirida3": "Encargo",
        "energia elet. adquirida3": "Encargo",
        "energia elet. adquirida 3* c/imposto": "Encargo",
        "energia elet. adquirida 3* c/lmposto": "Encargo",
        "energia elet. adquirida 3* c imp": "Encargo",
        "energia elet adquirida 3 c imposto": "Encargo",
        "Liminar Red Icms Fatura": "Encargo",
        "liminar red icms fatura": "Encargo",
        "liminarredicmsfatura": "Encargo",
        "liminar red. icms fatura": "Encargo",
        "Subtotal Faturamento": "Encargo",
        "subtotal faturamento": "Encargo",
        "sub total faturamento": "Encargo",
        "subtot. faturamento": "Encargo",
        "Componente Encargo kWh HFP": "Encargo",
        "componente encargo kwh hfp": "Encargo",
        "componente encargokwh hfp": "Encargo",
        "componente encargo kw hfp": "Encargo",
        "Componente Encargo kWh HP": "Encargo",
        "componente encargo kwh hp": "Encargo",
        "componente encargokwh hp": "Encargo",
        "componente encargo kw hp": "Encargo",
        "Componente Encargo HFP": "Encargo",
        "componente encargo hfp": "Encargo",
        "Componente Encargo HP": "Encargo",
        "componente encargo hp": "Encargo",
        "ENERGIA ACL": "Encargo",
        "energia acl": "Encargo",
        "energiaacl": "Encargo",
        "energia acl ": "Encargo",
        "CONSUMOATIVO PONTA TUSD": "Encargo",
        "consumoativo ponta tusd": "Encargo",
        "consumo ativo ponta tusd": "Encargo",
        "consumo ativoponta tusd": "Encargo",
        "CONSUMO ATIVO F.PONTA TUSD": "Encargo",
        "consumo ativo f.ponta tusd": "Encargo",
        "consumo ativo f. ponta tusd": "Encargo",
        "consumo ativo f ponta tusd": "Encargo",
        "BENEFICIO TARIFARIO BRUTO": "Encargo",
        "beneficio tarifario bruto": "Encargo",
        "benefício tarifario bruto": "Encargo",
        "beneficio tar. bruto": "Encargo",
        "benefício tar. bruto": "Encargo",
        "PIS/COFINS da subvencao/descon": "Encargo",
        "PIS/COFINS da subvencäo/descon": "Encargo",
        "pis cofins da subvençao/descon": "Encargo",
        "piscofins da subvencao descon": "Encargo",
        "Correcao IPCA/IGPM s/ conta 01/25 pg 19/02/25": "Encargo",
        "correção ipca igpm s/ conta": "Encargo",
        "correcao ipca igpm s/conta": "Encargo",
        "correcao ipca/igpm s/conta": "Encargo",
        "correcao ipca igpm s conta": "Encargo",
        "Componente Encargo kWh HFP": "Encargo",
        "componente encargo kwh hfp": "Encargo",
        "componente encargokwh hfp": "Encargo",
        "Componente Encargo kWh HP": "Encargo",
        "componente encargo kwh hp": "Encargo",
        "componente encargokwh hp": "Encargo",
        "PIS/COFINS da subvencao/descon": "Encargo",
        "PIS/COFINS da subvencäo/descon": "Encargo",
        "PIS/COFINS da subvenção/descon": "Encargo",
        "pis cofins da subvençao descon": "Encargo",
        "Encargo Cta Covid REN 885/2020": "Encargo",
        "encargo cta covid ren885/2020": "Encargo",
        "encargo covid ren 885/2020": "Encargo",
        "encargo covid ren885/2020": "Encargo",
        "Energia Ativa kWh HFP/nico": "Encargo",
        "energia ativa kwh hfp/unico": "Encargo",
        "energia ativa kwh hfp unico": "Encargo",
        "energia ativa kwh hfp": "Encargo",
        "Energia Ativa kWh HP": "Encargo",
        "energia ativa kwh hp": "Encargo",
        "DEBITO VAR IPCA": "Encargo",
        "débito var ipca": "Encargo",
        "debito var ipca": "Encargo",
        "deb1to var ipca": "Encargo",
        "ENERGIA ACL": "Encargo",
        "energia acl": "Encargo",
        "energiaacl": "Encargo",
        "TUSD ponta": "Encargo",
        "TUSD fora ponta": "Encargo",
        "tusd ponta": "Encargo",
        "tusd fora ponta": "Encargo",
        "Consumo Energia Elétrica HP": "Encargo",
        "consumo energia eletrica hp": "Encargo",
        "consumo energia elétrica hp": "Encargo",
        "DEMANDALEIESTADUAL16.886/18": "Encargo",




#===================================================================Desc. Encargo===================================================================
        "DEDUCAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCÃO ENERGIA ACL": "Desc. Encargo",
        "DED. ENERGIA ACL": "Desc. Encargo",
        "D. ENERGIA ACL": "Desc. Encargo",
        "Valor de Autoprodução HFP": "Desc. Encargo",
        "BENEFICIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "BENEFÍCIO TARIFÁRIO LIQUIDO": "Desc. Encargo",
        "BENEFiCIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "BENEFÍCIO TARIFÁRIO LÍQUIDO": "Desc. Encargo",
        "BENEFICIO TARIFÁRIO LÍQUIDO": "Desc. Encargo",
        "liminar icm5 tusd-consumo": "Desc. Encargo",
        "liminar icms tusd-consu mo": "Desc. Encargo",
        "liminar icms tusd-con5umo": "Desc. Encargo",
        "liminaricms tusd consumo": "Desc. Encargo",
        "liminar icms t usd-consumo": "Desc. Encargo",
        "liminar icms tu sd consumo": "Desc. Encargo",
        "liminar icms tusd-con sumo": "Desc. Encargo",
        "liminar icms t u s d-consumo": "Desc. Encargo",
        "liminar icms tusd-demanda": "Desc. Encargo",
        "liminar icms tusd demanda": "Desc. Encargo",
        "liminar icms tusddemanda": "Desc. Encargo",
        "liminar icms tus ddemanda": "Desc. Encargo",
        "liminar 1cms tusd-demanda": "Desc. Encargo",
        "liminar icms tuso-demanda": "Desc. Encargo",
        "liminar icm5 tusd-demanda": "Desc. Encargo",
        "liminar icms tusd demand a": "Desc. Encargo",
        "liminaricms tusd demanda": "Desc. Encargo",
        "liminar icms tu sd-demanda": "Desc. Encargo",
        "liminar icms t usd-demanda": "Desc. Encargo",
        "liminar icms tus-d-demanda": "Desc. Encargo",
        "liminar icms tusd d emanda": "Desc. Encargo",
        "liminar icms t u s d-demanda": "Desc. Encargo",
        "Liminar ICMS TUSD-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSD Demanda": "Desc. Encargo",
        "Liminar ICMS TUSDDemanda": "Desc. Encargo",
        "Liminar ICMS TUS D Demanda": "Desc. Encargo",
        "Liminar ICMS TUS-D-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSD Deman da": "Desc. Encargo",
        "Liminar 1CMS TUSD-Demanda": "Desc. Encargo",
        "Llminar ICMS TUSD-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSO-Demanda": "Desc. Encargo",
        "Liminar ICMS TUS-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSDDemand a": "Desc. Encargo",
        "Liminar ICMS TUSD-Consumo": "Desc. Encargo",
        "Liminar ICMS TUSD Consumo": "Desc. Encargo",
        "DEDUCAO ENERGIAACL": "Desc. Encargo",
        "DEDUÇÃO ENERGIAACL": "Desc. Encargo",
        "DED. ENERGIAACL": "Desc. Encargo",
        "deducao energiaacl": "Desc. Encargo",
        "deducaoenergiaacl":  "Desc. Encargo",
        "deducao energia acl": "Desc. Encargo",
        "DEDUÇÃO ENERGIA ACL": "Desc. Encargo",
        "Liminar ICMS TUSDConsumo": "Desc. Encargo",
        "Liminar ICMS TUS D Consumo": "Desc. Encargo",
        "Liminar ICMS TUSD-Consu mo": "Desc. Encargo",
        "Liminar ICMS TUSO-Consumo": "Desc. Encargo",
        "Liminar 1CMS TUSD-Consumo": "Desc. Encargo",
        "Llminar ICMS TUSD-Consumo": "Desc. Encargo",
        "Liminar ICMS TUSD-Consuno": "Desc. Encargo",
        "Liminar ICMS TUS DConsumo": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL ": "Desc. Encargo",
        "Valor de Autoproducao FP": "Desc. Encargo",
        "DEDUCAO ENERGIAACL": "Desc. Encargo",
        "DEDUTCAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA ACl": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL.": "Desc. Encargo",
        "DEDUTCAO ENERGIA ACL ": "Desc. Encargo",
        "DEDULCAO ENERGIA ACL": "Desc. Encargo",
        "DED0CAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA  ACL": "Desc. Encargo",
        "DEDUC0AO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA_ACL": "Desc. Encargo",
        "DED0CAO ENERGIA_ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL-": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL :": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL .": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL,": "Desc. Encargo",
        "Valor de Autoprodução HP": "Desc. Encargo",
        "BENEFiCIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "BENEFÍCIO TARIFÁRIO LIQUIDO": "Desc. Encargo",
        "BENEFÍCIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "BENEFICIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL ": "Desc. Encargo",
        "DEDUCAO ENERGIAACL": "Desc. Encargo",
        "DEDUCÃO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIAAC": "Desc. Encargo",
        "DEDUCAO ENERGIAA CL": "Desc. Encargo",
        "DEDUCAO ENERGIA-ACL": "Desc. Encargo",
        "DEDUCAO-ENERGIA ACL": "Desc. Encargo",
        "DEDUC0AO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAOENERGIA ACL": "Desc. Encargo",
        "DEDUTCAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL    ": "Desc. Encargo",
        "DEDUCAO ENERGIAACL.": "Desc. Encargo",
        "DEDUCAO ENERGIAACL:": "Desc. Encargo",
        "DED0CAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIA A C L": "Desc. Encargo",
        "DEDUCAO ENERGIAACL-": "Desc. Encargo",
        "DEDUCAO ENERGIAA-CL": "Desc. Encargo",
        "DEDUCAO ENERGIAACL ,": "Desc. Encargo",
        "Desconto Comp. Encargo HP": "Desc. Encargo",
        "Ajuste de Desconto C. Enc HP": "Desc. Encargo",
        "Desconto Comp. Encargo HFP": "Desc. Encargo",
        "Ajuste de Desconto C. Enc HFP": "Desc. Encargo",
        "desconto compencargo hp": "Desc. Encargo",
        "ajuste de desconto cenc hp": "Desc. Encargo",
        "Valor Autoprodução HFP": "Desc. Encargo",
        "Valor de Autoproducao HFP": "Desc. Encargo",
        "Valor Autoprodução HP": "Desc. Encargo",
        "Valor de Autoproducao HP": "Desc. Encargo",
        "Desconto Comp Encargo HP": "Desc. Encargo",
        "Desconto Compencargo HP": "Desc. Encargo",
        "Ajuste de Desconto C Enc HP": "Desc. Encargo",
        "Ajuste de Desconto CEnc HP": "Desc. Encargo",
        "Ajuste de Desconto C Enc HFP": "Desc. Encargo",
        "Ajuste de Desconto CEnc HFP": "Desc. Encargo",
        "Desconto Comp Encargo HFP": "Desc. Encargo",
        "Desconto Compencargo HFP": "Desc. Encargo",
        "Valor de Autoproducao FP": "Desc. Encargo",
        "Valor de Autoprodução FP": "Desc. Encargo",
        "Valor de Autoproduç:ao FP": "Desc. Encargo",
        "Valor de Autoproducäo FP": "Desc. Encargo",
        "Valor Autoproducao FP": "Desc. Encargo",
        "Valor de Autoproducao F P": "Desc. Encargo",
        "Autoproducao FP": "Desc. Encargo",
        "Valor Autoproducao": "Desc. Encargo",
        "Restituicao de Pagamento": "Desc. Encargo",
        "Restituição de Pagamento": "Desc. Encargo",
        "Restituic:ao de Pagamento": "Desc. Encargo",
        "Restitüição de Pagamento": "Desc. Encargo",
        "Restituicaoo de Pagamento": "Desc. Encargo",
        "Restituicão Pagamento": "Desc. Encargo",
        "liminar icms tusd-consumo": "Desc. Encargo",
        "liminar icms tusd consumo": "Desc. Encargo",
        "liminar icms tusdconsumo": "Desc. Encargo",
        "liminar icms tus dconsumo": "Desc. Encargo",
        "liminar 1cms tusd-consumo": "Desc. Encargo",
        "liminar icm5 tusd-consumo": "Desc. Encargo",
        "liminar icms tusd-consu mo": "Desc. Encargo",
        "liminar icms tusd-con5umo": "Desc. Encargo",
        "liminaricms tusd consumo": "Desc. Encargo",
        "liminar icms t usd-consumo": "Desc. Encargo",
        "liminar icms tu sd consumo": "Desc. Encargo",
        "liminar icms tusd-con sumo": "Desc. Encargo",
        "liminar icms t u s d-consumo": "Desc. Encargo",
        "liminar icms tusd-demanda": "Desc. Encargo",
        "liminar icms tusd demanda": "Desc. Encargo",
        "liminar icms tusddemanda": "Desc. Encargo",
        "liminar icms tus ddemanda": "Desc. Encargo",
        "liminar 1cms tusd-demanda": "Desc. Encargo",
        "liminar icms tuso-demanda": "Desc. Encargo",
        "liminar icm5 tusd-demanda": "Desc. Encargo",
        "liminar icms tusd demand a": "Desc. Encargo",
        "liminaricms tusd demanda": "Desc. Encargo",
        "liminar icms tu sd-demanda": "Desc. Encargo",
        "liminar icms t usd-demanda": "Desc. Encargo",
        "liminar icms tus-d-demanda": "Desc. Encargo",
        "liminar icms tusd d emanda": "Desc. Encargo",
        "liminar icms t u s d-demanda": "Desc. Encargo",
        "Liminar ICMS TUSD-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSD Demanda": "Desc. Encargo",
        "Liminar ICMS TUSDDemanda": "Desc. Encargo",
        "Liminar ICMS TUS D Demanda": "Desc. Encargo",
        "Liminar ICMS TUS-D-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSD Deman da": "Desc. Encargo",
        "Liminar 1CMS TUSD-Demanda": "Desc. Encargo",
        "Llminar ICMS TUSD-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSO-Demanda": "Desc. Encargo",
        "Liminar ICMS TUS-Demanda": "Desc. Encargo",
        "Liminar ICMS TUSDDemand a": "Desc. Encargo",
        "Liminar ICMS TUSD-Consumo": "Desc. Encargo",
        "Liminar ICMS TUSD Consumo": "Desc. Encargo",
        "Liminar ICMS TUSDConsumo": "Desc. Encargo",
        "Liminar ICMS TUS D Consumo": "Desc. Encargo",
        "Liminar ICMS TUSD-Consu mo": "Desc. Encargo",
        "Liminar ICMS TUSO-Consumo": "Desc. Encargo",
        "Liminar 1CMS TUSD-Consumo": "Desc. Encargo",
        "Llminar ICMS TUSD-Consumo": "Desc. Encargo",
        "Liminar ICMS TUSD-Consuno": "Desc. Encargo",
        "Liminar ICMS TUS DConsumo": "Desc. Encargo",
        "Valor de Autoproducao FP": "Desc. Encargo",
        "Valor de Autoprodução FP": "Desc. Encargo",
        "Valor de Autoproduç:ao FP": "Desc. Encargo",
        "Valor de Autoproducäo FP": "Desc. Encargo",
        "Valor Autoproducao FP": "Desc. Encargo",
        "Valor de Autoproducao F P": "Desc. Encargo",
        "Autoproducao FP": "Desc. Encargo",
        "Valor Autoproducao": "Desc. Encargo",
        "Restituicao de Pagamento": "Desc. Encargo",
        "Restituição de Pagamento": "Desc. Encargo",
        "Restituic:ao de Pagamento": "Desc. Encargo",
        "Restitüição de Pagamento": "Desc. Encargo",
        "Restituicaoo de Pagamento": "Desc. Encargo",
        "Restituicão Pagamento": "Desc. Encargo",
        "Valor de Autoprodução FP": "Desc. Encargo",
        "Valor de Autoproducao FP": "Desc. Encargo",
        "Valor de Autoproducão FP": "Desc. Encargo",
        "Valor de Autoproducäo FP": "Desc. Encargo",
        "Valor de Autoprodução HFP": "Desc. Encargo",
        "Valor de Autoproducao HFP": "Desc. Encargo",
        "Valor de Autoproducão HFP": "Desc. Encargo",
        "Valor de Autoproducäo HFP": "Desc. Encargo",
        "Valor de Autoprodução HP": "Desc. Encargo",
        "Valor de Autoproducao HP": "Desc. Encargo",
        "Valor Autoproducao FP": "Desc. Encargo",
        "Valor Autoproducao HFP": "Desc. Encargo",
        "Valor Autoproducao HP": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL": "Desc. Encargo",
        "DEDUÇÃO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIAACL": "Desc. Encargo",
        "DEDUCAO ENERGIAA CL": "Desc. Encargo",
        "DEDUCAOENERGIA ACL": "Desc. Encargo",
        "DEDUCÃO ENERGIA ACL": "Desc. Encargo",
        "BENEFICIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "BENEFÍCIO TARIFÁRIO LIQUIDO": "Desc. Encargo",
        "BENEFiCIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "BENEFÍCIO TARIFÁRIO LÍQUIDO": "Desc. Encargo",
        "Desconto Comp.Encargo HP": "Desc. Encargo",
        "Desconto Comp Encargo HP": "Desc. Encargo",
        "Desconto Compencargo HP": "Desc. Encargo",
        "Ajuste de Desconto C.Enc HP": "Desc. Encargo",
        "Ajuste de Desconto C Enc HP": "Desc. Encargo",
        "Ajuste de Desconto CEnc HP": "Desc. Encargo",
        "Desconto Comp.Encargo HFP": "Desc. Encargo",
        "Ajuste de Desconto C.Enc HFP": "Desc. Encargo",
        "Ajuste de Desconto C Enc HFP": "Desc. Encargo",
        "Ajuste de Desconto CEnc HFP": "Desc. Encargo",
        "Restituicao de Pagamento": "Desc. Encargo",
        "Restituição de Pagamento": "Desc. Encargo",
        "TAR.REDUZ.RES.ANEEL166/05P": "Desc. Encargo",
        "TAR.REDUZ.RES.ANEEL166/05FP": "Desc. Encargo",
        "Diferenca Energia Retroativa": "Desc. Encargo",
        "Desconto Comp.Encargo HP": "Desc. Encargo",
        "Ajuste de Desconto C.Enc HP": "Desc. Encargo",
        "Ajuste de Desconto C.Enc HFP": "Desc. Encargo",
        "Valor de Autoprodução FP": "Desc. Encargo",
        "Valor de Autoproducao FP": "Desc. Encargo",
        "Valor de Autoprodução HFP": "Desc. Encargo",
        "Valor de Autoproducao HFP": "Desc. Encargo",
        "Valor de Autoprodução HP": "Desc. Encargo",
        "Valor de Autoproducao HP": "Desc. Encargo",
        "DEDUCAO ENERGIA ACL": "Desc. Encargo",
        "DEDUCAO ENERGIAACL": "Desc. Encargo",
        "BENEFICIO TARIFARIO LIQUIDO": "Desc. Encargo",
        "TAR.REDUZ.RES.ANEEL166/05P": "Desc. Encargo",
        "TAR.REDUZ.RES.ANEEL166/05FP": "Desc. Encargo",
        "Restituicao de Pagamento": "Desc. Encargo",
        "Diferenca Energia Retroativa": "Desc. Encargo",
        "Energia Terc Comercializad HP": "Desc. Encargo",
        "Energia Terc Comercializad HFP": "Desc. Encargo",

#===================================================================Demanda===================================================================
        "demanda ponta c/desconto": "Demanda",
        "demanda ponta cidesconto": "Demanda",
        "demanda ponta cidescon": "Demanda",
        "demanda ponta c/desc0nt0": "Demanda",
        "DEMANDA FORA PONTA C/DESC": "Demanda",
        "DEMANDA FORA PONTA C/DESC0": "Demanda",
        "DEMANDA FORA PONTA C/DESCONTO": "Demanda",
        "DEMANDA FORA PONTA C/DESC0NT0": "Demanda",
        "DEMANDA FORA PONTA CIDESCONTO": "Demanda",
        "DEMANDA FORA PONTA CIDESC0NTO": "Demanda",
        "demanda p0nta c/desconto": "Demanda",
        "demanda ponta c / desconto": "Demanda",
        "ULTRAPASSAGEM PONTA/TUSD": "Demanda",
        "ULTRAPASSAGEM PONTA TUSD": "Demanda",
        "ULTRAPASSAGEM PONTA TUSD-": "Demanda",
        "ULTRAPASSAGEM PONTA/TUSD ": "Demanda",
        "ULTRAPASSAGEM PONTA/T USD": "Demanda",
        "ULTRAPASSAGEM PONTA/TUSD-": "Demanda",
        "ULTRAPASSAGEM PONTA/TUSD  ": "Demanda",
        "ULTRAPASSAGEM PONTA-TUSD": "Demanda",
        "ULTRA PASSAGEM PONTA/TUSD": "Demanda",
        "ULTRA PASSAGEM PONTA TUSD": "Demanda",
        "ULTRAPASSAGEM PONTA TUSD ": "Demanda",
        "ULTRAPASSAGEM FORAPONTA/TUSD": "Demanda",
        "ULTRAPASSAGEM FORA PONTA TUSD": "Demanda",
        "ULTRAPASSAGEM FORAPONTA TUSD": "Demanda",
        "ULTRAPASSAGEM FORA PONTA/TUSD": "Demanda",
        "ULTRAPASSAGEM FORA PONTA TUSD ": "Demanda",
        "UFDR PONTA/TUSD": "Demanda",
        "UFDRPONTA/TUSD": "Demanda",
        "UFDR PONTA TUSD": "Demanda",
        "UFDRPONTATUSD": "Demanda",
        "UFDR P0NTA/TUSD": "Demanda",
        "UFDR PONTA/T USD": "Demanda",
        "UFDR P0NTA TUSD": "Demanda",
        "UFDR PONTA-TUSD": "Demanda",
        "UFDR PONTA/TUS D": "Demanda",
        "UFDRFORA PONTA/TUSD": "Demanda",
        "UFDR FORA PONTA/TUSD": "Demanda",
        "UFDR FORAPONTA/TUSD": "Demanda",
        "UFDR FORA PONTA TUSD": "Demanda",
        "UFDRFORAPONTA TUSD": "Demanda",
        "UFDRFORA PONTA TUSD": "Demanda",
        "UFDR FORA P0NTA/TUSD": "Demanda",
        "UFDRFORA PONTA-TUSD": "Demanda",
        "UFDR FORA PONTA/TUS D": "Demanda",
        "ULTRAPASSAGEM FORA PONTA T USD": "Demanda",
        "ULTRAPASSAGEM FORA PONTA TUSD-": "Demanda",
        "ULTRAPASSAGEM FORA PONTA/TUSD ": "Demanda",
        "ULTRAPASSAGEMFORAPONTA TUSD": "Demanda",
        "ULTRAPASSAGEMF.PONTA/TUSD": "Demanda",
        "ULTRAPASSAGEM FORA PONTA/TUSD-": "Demanda",
        "ULTRA PASSAGEM FORA PONTA/TUSD": "Demanda",
        "demanda pontac/desconto": "Demanda",
        "demanda ponta c/desconto": "Demanda",
        "demanda ponta cidesconto": "Demanda",
        "demanda ponta cidescon": "Demanda",
        "demanda ponta c/desc0nt0": "Demanda",
        "Componente Fio kW HFP": "Demanda",
        "Componente Fio kW HP": "Demanda",
        "Component Fio kW HFP": "Demanda",
        "Component Fio kW HP": "Demanda",
        "Componente Fio kWHP": "Demanda",
        "Componente Fio KW HP": "Demanda",
        "Componente Fio KW HFP": "Demanda",
        "Componete Fio kW HFP": "Demanda",
        "Comp Fio kW HFP": "Demanda",
        "Comp Fio kW HP": "Demanda",
        "Componente Fio kw hfp": "Demanda",
        "demanda p0nta c/desconto": "Demanda",
        "demanda ponta c / desconto": "Demanda",
        "demanda f0ra p0nta cidescont0": "Demanda",
        "demanda fora ponta cidesconto": "Demanda",
        "demanda fora p0nta cidesconto": "Demanda",
        "demanda fora ponta cidesc0nto": "Demanda",
        "demanda fora ponta cidescon": "Demanda",
        "demanda f ora ponta cidesconto": "Demanda",
        "demanda fora pontacidesconto": "Demanda",
        "DEMANDA PONTA C/DESCONTO": "Demanda",
        "DEMANDA PONTA C DESCONTO": "Demanda",
        "DEMANDA PONTA C/DESC0NT0": "Demanda",
        "DEMANDA P0NTA C/DESCONTO": "Demanda",
        "DEMANDA P0NTA C DESCONTO": "Demanda",
        "DEMANDA FORA PONTA CIDESCONTO": "Demanda",
        "DEMANDA F0RA P0NTA CIDESCONTO": "Demanda",
        "DEMANDA FORA PONTA C/DESC": "Demanda",
        "DEMANDA  FORA  PONTA  C/DESC": "Demanda",
        "DEMANDA FORA PONTA C/DESC0NT0": "Demanda",
        "Consumo Energia Eletrica HFP": "Demanda",
        "Consumo de Energia Eletrica HFP": "Demanda",
        "Consumo Energia Eletrica HFP/Unico": "Demanda",
        "Consumo de Energia Elétrica HFP": "Demanda",
        "Consumo Energia Elétric HFP": "Demanda",
        "Consumo Energia Eléctrica HFP": "Demanda",
        "Consumo Energia Eletrica kWh HFP": "Demanda",
        "Componente Fio HFP s/ ICMS": "Demanda",
        "Componente Fio HFP": "Demanda",
        "Componente Fio HP": "Demanda",
        "Componente Fio HFP s/ICMS": "Demanda",
        "Componente Fio kW HFP": "Demanda",
        "Componente Fio kW HP": "Demanda",
        "Component Fio kW HFP": "Demanda",
        "Component Fio kW HP": "Demanda",
        "Componente Fio kWHP": "Demanda",
        "Componente Fio KW HP": "Demanda",
        "Componente Fio KW HFP": "Demanda",
        "Componete Fio kW HFP": "Demanda",
        "Comp Fio kW HFP": "Demanda",
        "Comp Fio kW HP": "Demanda",
        "Componente Fio kw hfp": "Demanda",
        "Demanda Ativa kW HFP/Unico": "Demanda",
        "Demanda Ativa kW HFP/Único": "Demanda",
        "Demanda Ativa kW HFP/ünico": "Demanda",
        "Demanda Ativa kW HFP/Un1co": "Demanda",
        "Demanda Ativa kW HFP/Unic0": "Demanda",
        "Demanda AtivakW HFP/Unico": "Demanda",
        "Demanda Ativa kWHFP/Unico": "Demanda",
        "Componente Fio kW HFP": "Demanda",
        "Componente Fio kW HP": "Demanda",
        "Component Fio kW HFP": "Demanda",
        "Component Fio kW HP": "Demanda",
        "Componente Fio kWHP": "Demanda",
        "Componente Fio KW HP": "Demanda",
        "Componente Fio KW HFP": "Demanda",
        "Componete Fio kW HFP": "Demanda",
        "Comp Fio kW HFP": "Demanda",
        "Comp Fio kW HP": "Demanda",
        "Componente Fio kw hfp": "Demanda",
        "Componente Fio HFP": "Demanda",
        "Componente Fio HP": "Demanda",
        "Componente Fio kW HP": "Demanda",
        "Componente Fio HFP s/ICMS": "Demanda",
        "Componente Fio HP s/ICMS": "Demanda",
        "Componente Fio HP s/ ICMS": "Demanda",
        "Componente Fio HFP s/ ICMS": "Demanda",
        "DEMANDA PONTA C/DESCONTO": "Demanda",
        "DEMANDA FORA PONTA CIDESCONTO": "Demanda",
        "DEMANDA UNICA C/DESCONTO": "Demanda",
        "Demanda Distribuicao F. Ponta": "Demanda",
        "Demanda Distribuicao Ponta": "Demanda",
        "Dernanda Distribuicao Ponta": "Demanda",
        "DIF.FATUR.TUSD-HOMOLOG.CCEE": "Demanda",
        "ULTRAPASSAGEM FORAPONTA/TUSD": "Demanda",




#===================================================================DescDemanda===================================================================
        "DIF.FATUR.TUSD-HOMOLOG.CCEE": "Desc. Demanda",
        "DIF.FATUR.TUSD-HOMOLOG.C0EE": "Desc. Demanda",
        "DIF.FATUR.TUSD-HOMOLOG.0CEE": "Desc. Demanda",
        "DIF.FATUR.TUSD HOMOLOG CCEE": "Desc. Demanda",
        "DIF.FATUR TUSD-HOMOLOG CCEE": "Desc. Demanda",
        "DIF.FATUR.T0SD-HOMOLOG.CCEE": "Desc. Demanda",
        "D1F.FATUR.TUSD-H0MOLOG.CCEE": "Desc. Demanda",
        "DIF.FATUR.TUSD HOMOLOG.CCEE": "Desc. Demanda",
        "DIF FATUR.TUSD-HOMOLOG.CCEE": "Desc. Demanda",
        "DIF.FATUR.TUSD-HOMOLOG.C0E": "Desc. Demanda",
        "DIF.FATUR.TUSD-HOMOLOG.CEE": "Desc. Demanda",
        "DIF.FATUR.TUSD-HOMOLOG.CCE": "Desc. Demanda",
        "ajuste de desconto cfio hfp": "Desc. Demanda",
        "desconto compfio hfp": "Desc. Demanda",
        "Ajuste de Desconto C Fio HFP": "Desc. Demanda",
        "Ajuste de Desconto CFio HFP": "Desc. Demanda",
        "Ajuste de Desconto cfio hfp": "Desc. Demanda",
        "Desconto Comp Fio HFP": "Desc. Demanda",
        "Desconto Compfio HFP": "Desc. Demanda",
        "Ajuste de Desconto C Fio HP": "Desc. Demanda",
        "Ajuste de Desconto CFio HP": "Desc. Demanda",
        "Desconto Comp Fio HP": "Desc. Demanda",
        "Desconto Compfio HP": "Desc. Demanda",
        "Ajuste de Desconto C. Fio HFP": "Desc. Demanda",
        "Ajuste de Desconto C. Fio HP": "Desc. Demanda",
        "Desconto Comp. Fio HFP": "Desc. Demanda",
        "Desconto Comp. Fio HP": "Desc. Demanda",
        "Desconto Comp.Fio HP": "Desc. Demanda",
        "Desconto Comp Fio HP": "Desc. Demanda",
        "Desc. Comp. Fio HP": "Desc. Demanda",
        "Desc Comp.Fio HP": "Desc. Demanda",
        "Desconto Compfio HP": "Desc. Demanda",
        "Desconto Comp FioHP": "Desc. Demanda",
        "Desconto Componente Fio HP": "Desc. Demanda",
        "Desconto Comp.Fio HFP": "Desc. Demanda",
        "Desconto Comp Fio HFP": "Desc. Demanda",
        "Desc. Comp. Fio HFP": "Desc. Demanda",
        "Desc Comp.Fio HFP": "Desc. Demanda",
        "Desconto Compfio HFP": "Desc. Demanda",
        "Desconto Comp FioHFP": "Desc. Demanda",
        "Desconto Componente Fio HFP": "Desc. Demanda",
        "Ajuste de Desconto C.Fio HP": "Desc. Demanda",
        "Ajuste de Desconto C. Fio HP": "Desc. Demanda",
        "Ajuste de Desconto CFio HP": "Desc. Demanda",
        "Ajute de Desconto C.Fio HP": "Desc. Demanda",
        "Ajuste de Desc. C. Fio HP": "Desc. Demanda",
        "Ajuste Desc C.Fio HP": "Desc. Demanda",
        "Ajuste de Desconto C fio HP": "Desc. Demanda",
        "Ajuste Desconto C.Fio HP": "Desc. Demanda",
        "Ajuste de Desconto C Fio HP": "Desc. Demanda",
        "Ajuste de Desconto Cfio HP": "Desc. Demanda",
        "Ajuste de Desconto C.Fio HFP": "Desc. Demanda",
        "Ajuste de Desconto C. Fio HFP": "Desc. Demanda",
        "Ajuste de Desconto CFio HFP": "Desc. Demanda",
        "Ajute de Desconto C.Fio HFP": "Desc. Demanda",
        "Ajste de Desconto C.Fio HFP": "Desc. Demanda",
        "Ajuste de Desc. C. Fio HFP": "Desc. Demanda",
        "Ajuste Desc C.Fio HFP": "Desc. Demanda",
        "Ajuste de Desconto Cfio HFP": "Desc. Demanda",
        "Ajuste de Desconto C fio HFP": "Desc. Demanda",
        "DIF.FATUR.TUSD HOMOLOGACAO CCEE": "Desc. Demanda",
        "DIF.FATUR.TUSD-HOMOLOGACAO CCEE": "Desc. Demanda",
        "DIF.FATUR TUSD HOMOLOGACAO CCEE": "Desc. Demanda",
        "DIF. FATUR. TUSD HOMOLOG CCEE": "Desc. Demanda",
        "DIF FATUR TUSD HOMOLOGACAO CCEE": "Desc. Demanda",
        "DIF FATUR TUSD-HOMOLOGACAO CCEE": "Desc. Demanda",
        "DIF. FATURACAO TUSD CCEE": "Desc. Demanda",
        "DIFERENCA FATURAMENTO TUSD CCEE": "Desc. Demanda",
        "D1F.FATUR.TUSD-H0MOLOG.C0EE": "Desc. Demanda",
        "DIF.FATUR.T0SD HOMOLOG CCE": "Desc. Demanda",
        "DIF.FATUR.TUSD HOM0LOG CCE": "Desc. Demanda",
        "D1F FATURACAO TUSD CCEE": "Desc. Demanda",
        "AJUSTE DESC COMP FIO HP": "Desc. Demanda",
        "AJUSTE DESC. COMP. FIO HFP": "Desc. Demanda",
        "AJUSTE DESC. COMP. FIO HP": "Desc. Demanda",
        "AJUSTE DESCONTO COMP FIO HP": "Desc. Demanda",
        "DESC. COMPONENTE FIO HP": "Desc. Demanda",
        "DESC. COMPONENTE FIO HFP": "Desc. Demanda",
        "AJSTE DE DESCONTO C. FIO HP": "Desc. Demanda",
        "AJSTE DE DESCONTO C. FIO HFP": "Desc. Demanda",
        "DESC COMP. FIO HFP": "Desc. Demanda",
        "DESC COMP. FIO HP": "Desc. Demanda",
        "DESC COMP.FIO HP": "Desc. Demanda",
        "AJUSTEDESCONTOCFIOHFP": "Desc. Demanda",
        "DESC.COMPFIOHFP": "Desc. Demanda",
        "AJUSTEDESC.C.FIOHP": "Desc. Demanda",




#===================================================================Reativa===================================================================
        "UFER PONTA TE": "Reativa",
        "UFERFORA PONTA TE": "Reativa",
        "UFER PONTA  TE": "Reativa",
        "UFER PONTA TE ": "Reativa",
        "UFERF0RA PONTA TE": "Reativa",
        "UFER P0NTA TE": "Reativa",
        "UFER PONTA  TEE": "Reativa",
        "UFER PONTA-TE": "Reativa",
        "UFER PONTA.TE": "Reativa",
        "UFER F0RA PONTA TE": "Reativa",
        "UFFER PONTA TE": "Reativa",
        "UFFER FORA PONTA TE": "Reativa",
        "UFER PONTA_Te": "Reativa",
        "UFER  PONTA TE": "Reativa",
        "UFER PONTA - TE": "Reativa",
        "UFER PONTA_Te": "Reativa",
        "UFER- PONTA TE": "Reativa",
        "UFERPONTA TE": "Reativa",
        "UFER P0NTA TE ": "Reativa",
        "UFER PONTA TE": "Reativa",
        "Energia Reativa HFP": "Reativa",
        "Energia Reativa HP": "Reativa",
        "UFERFORA PONTA TE": "Reativa",
        "UFER PONTA  TE": "Reativa",
        "UFER PONTA TE ": "Reativa",
        "UFERF0RA PONTA TE": "Reativa",
        "UFER P0NTA TE": "Reativa",
        "UFER PONTA  TEE": "Reativa",
        "UFER PONTA-TE": "Reativa",
        "UFER PONTA.TE": "Reativa",
        "UFER F0RA PONTA TE": "Reativa",
        "Consumo Reativo Excedente Fp": "Reativa",
        "Consumo Reativo Excedente Np": "Reativa",
        "UFER FP/único (não Faturado)": "Reativa",
        "UFER FP/unico (nao Faturado)": "Reativa",
        "ufer fp/unico (nao faturado)": "Reativa",
        "ufer fp/unico (não faturado)": "Reativa",
        "ufer fp/único (nao faturado)": "Reativa",
        "ufer fp unico (nao faturado)": "Reativa",
        "uferfp/unico (nao faturado)": "Reativa",
        "ufer fp/ünico (nao faturado)": "Reativa",
        "ufer fp/ünico (n40 faturado)": "Reativa",
        "ufer fp/unico (n4o faturado)": "Reativa",
        "ufer fp/unico nao faturado": "Reativa",
        "ufer fp unico nao faturado": "Reativa",
        "ufer fp/nico (nao faturado)": "Reativa",
        "ufer fp/unico (não fatvrado)": "Reativa",
        "ufer fp/un1co (n4o faturado)": "Reativa",
        "ufer fp/un1co (nao faturado)": "Reativa",
        "ufer fp/unico (n faturado)": "Reativa",
        "ufer fp/unico (nao faturad0)": "Reativa",
        "ufer fp/ünico nao faturado": "Reativa",
        "uferfp unico nao faturado": "Reativa",
        "ufer fp/único (nao fat)": "Reativa",
        "ufer fp unico (não fat)": "Reativa",
        "UFER FP/ünico (näo Faturado)": "Reativa",
        "UFER FP/un1co (n40 Faturado)": "Reativa",
        "UFER FP/unico (n Faturado)": "Reativa",
        "UFER FP unico nao Faturado": "Reativa",
        "UFER FP/único nao Faturado": "Reativa",
        "UFER FP/nico (nao Faturado)": "Reativa",
        "Energia Reativa kWh HFP/Unico": "Reativa",
        "Energia Reativa kWh HFP/Único": "Reativa",
        "Energia Reativa kWh HFP/unico": "Reativa",
        "Energia Reativa kWh HFP/único": "Reativa",
        "Energia Reativa kWh HFP": "Reativa",
        "Energia Reativa kWhHFP/Unico": "Reativa",
        "Energia Reativa kWh HFP Unico": "Reativa",
        "Energia Reativa kwh hfp unico": "Reativa",
        "Energia Reativa kwh hfp/nico": "Reativa",
        "Energia Reativa kwh HFP/ni co": "Reativa",
        "Energia Reativa kwhHFP/nico": "Reativa",
        "Energia Reativa kwhhfp": "Reativa",
        "Energia Reativa kWh HFPUnico": "Reativa",
        "Energia Reatíva kWh HFP/Unico": "Reativa",
        "Energia Reativa kWh HFP/U1nico": "Reativa",
        "Energia Reativa kWh HFP/Un1co": "Reativa",
        "Energia Reativa kWh HFP Ünico": "Reativa",
        "Energia Reativa kWh HFP ùnico": "Reativa",
        "Energia Reativa kWh HP": "Reativa",
        "Energia Reativa kwh HP": "Reativa",
        "Energia Reativa kWhHP": "Reativa",
        "Energia Reativa kwH HP": "Reativa",
        "Energia Reatíva kWh HP": "Reativa",
        "Energia Reativa kWh H P": "Reativa",
        "Energia Reativa KWh HP": "Reativa",
        "Energia Reativa kwh h p": "Reativa",
        "EnergiaReativa kWh HP": "Reativa",
        "Energia Reativa kwHP": "Reativa",
        "Energia Reativa kwh": "Reativa",
        "UFER FP UNICO": "Reativa",
        "UFER FP ÚNICO": "Reativa",
        "UFER FP/UNICO": "Reativa",
        "UFER FP/UN1CO": "Reativa",
        "UFERFP/UNICO": "Reativa",
        "UFER FP UNICO NAO FATURADO": "Reativa",
        "UFER FP UNICO (NAO FATURADO)": "Reativa",
        "UFER FP ÚNICO (NÃO FATURADO)": "Reativa",
        "UFERFP UNICO (NAO FATURADO)": "Reativa",
        "UFER FPUNICO (NAO FATURADO)": "Reativa",
        "UFER FP/NICO (NAO FATURADO)": "Reativa",
        "UFER FP/NlCO (NAO FATURADO)": "Reativa",
        "UFER FP/ÜNICO (NAO FATURADO)": "Reativa",
        "UFER FP/UNIC0 (NAO FATURADO)": "Reativa",
        "UFER FP/UN1CO NAO FATURADO": "Reativa",
        "ENERGIA REATIVA HFP": "Reativa",
        "ENERGIA REATIVA HP": "Reativa",
        "ENERGIA REATIVA KWH HFP": "Reativa",
        "ENERGIA REATIVA KWH HP": "Reativa",
        "ENERGIA REATIVA KWHHFP": "Reativa",
        "ENERGIA REATIVA KWHHP": "Reativa",
        "ENERGIAREATIVA KWH HP": "Reativa",
        "ENERGIA REATIVA KWHP": "Reativa",
        "ENERGIA REATIVA KWH HFP UNICO": "Reativa",
        "ENERGIA REATIVA KWH HFP/UNICO": "Reativa",
        "ENERGIA REATIVA KWH HFP/ÚNICO": "Reativa",
        "ENERGIA REATIVA KWH HFP/NICO": "Reativa",
        "ENERGIA REATIVA KWH HFP/ÜNICO": "Reativa",
        "ENERGIA REATIVA KWH HFP/UN1CO": "Reativa",
        "ENERGIA REATIVA KWH HFP/U1NICO": "Reativa",
        "ENERGIA REATIVA KWH HFP/UNCO": "Reativa",
        "ENERGIA REATIVA KWH H P": "Reativa",
        "ENERGIA REATIVA KWH HFPUNICO": "Reativa",
        "ENERGIA REATIVA KW HFP": "Reativa",
        "ENERGIA REATIVA KWH": "Reativa",
        "ENERGIAREATIVA KWH": "Reativa",
        "ENERGIA REATIVA HP ": "Reativa",
        "UFER PONTA TEE": "Reativa",
        "UFER FORA PONTA TE": "Reativa",
        "UFER FORA P0NTA TE": "Reativa",
        "UFER FORA PONTA-TE": "Reativa",
        "UFER FORA PONTA.TE": "Reativa",
        "UFER-FORA PONTA TE": "Reativa",
        "UFER PONTA TE (NÃO FATURADO)": "Reativa",



#===================================================================Cont Publica===================================================================
        "CIP-ILUM PUB PREF MUNICIPAL": "Contb. Pública",
        "CIP - ILUM PUB PREF MUNICIPAL": "Contb. Pública",
        "COSIP.SAOPAULO-MUNICIPAL": "Contb. Pública",
        "CIP-BARUERI-MUNICIPAL": "Contb. Pública",
        "CIP-CAJAMAR-MUNICIPAL": "Contb. Pública",
        "CIP-ILUM PUB PREF MUNICIPALCONTRIB PUB": "Contb. Pública",
        "CIP-SAOPAULO-MUNICIPAL": "Contb. Pública",
        "CIP-ILUM PUB PREF MUNICIPAL-SAO PAULO": "Contb. Pública",
        "CIP_ILUM_PUB_MUNICIPAL": "Contb. Pública",
        "Contrib llum Publica Municipal": "Contb. Publica",
        "Contrib Ilum Publica Municipal": "Contb. Publica",
        "Contrib Ilum Pública Municipal": "Contb. Publica",
        "Contrib IlumPublica Municipal": "Contb. Publica",
        "Contrib IlumPublicaMunicipal": "Contb. Publica",
        "Contrib llumPublica Municipal": "Contb. Publica",
        "Contrib llumPublicaMunicipal": "Contb. Publica",
        "CIPILUM PUB PREF MUNICIPAL": "Contb. Pública",
        "CIP ILUM PUB PREF MUNICIPAL": "Contb. Pública",
        "C1P - ILUM PUB PREF MUNICIPAL": "Contb. Pública",
        "C1P-ILUM PUB PREF MUNICIPAL": "Contb. Pública",
        "CIP ILUM PUBL PREF MUNICIPAL": "Contb. Pública",
        "COSIP SAOPAULO MUNICIPAL": "Contb. Pública",
        "COSIP SAO PAULO MUNICIPAL": "Contb. Pública",
        "C0SIP.SAOPAULO-MUNICIPAL": "Contb. Pública",
        "CIP-BARUERI MUNICIPAL": "Contb. Pública",
        "CIP CAJAMAR MUNICIPAL": "Contb. Pública",
        "CIP ILUM PUB MUNICIPAL": "Contb. Pública",
        "CIP-ILUM PUB PREF MUNICIPAL CONTRIB PUBLICA": "Contb. Pública",
        "CIP ILUM PUBLICA MUNICIPAL": "Contb. Pública",
        "CIP-ILUM PUBLICA MUNICIPAL": "Contb. Pública",
        "Contrib llum Publica Municpal": "Contb. Pública",
        "Contrib llum Púbica Municipal": "Contb. Pública",
        "Contrib Ilum Pública Municpal": "Contb. Pública",
        "Contribuçao Ilum Publica": "Contb. Pública",
        "Contribuiçao Ilum Pública": "Contb. Pública",
        "Contribuiçâo llum Pública Municipal": "Contb. Pública",
        "Cont llum Pública": "Contb. Pública",
        "Contrib IlumPublicaMunicipal": "Contb. Pública",
        "Contr llumPublicaMunicipal": "Contb. Pública",
        "Contribuicao Ilum Publica": "Contb. Pública",
        "Contribuicao IlumPublica": "Contb. Pública",
        "Contribuição Ilum Publica Municipal": "Contb. Pública",
        "Cont Ilum Publica": "Contb. Pública",
        "C0ntrib Ilum Publica": "Contb. Pública",
        "Contrib llum Püblica Municipal": "Contb. Publica",
        "Contribuição Iluminação Pública": "Contb. Publica",
        "Cont llum Publica": "Contb. Publica",
        "Contribuição Pública": "Contb. Publica",
        "Contrib llum Pública": "Contb. Publica",
        "Contrib IlumMunicipal": "Contb. Publica",
        "Contrib llum Püblica Municipal": "Contb. Publica",
        "Contrib Ilum Pública Municipal": "Contb. Publica",
        "Contrib Ilum Publica Municipal": "Contb. Publica",
        "Contrib IlumPublica Municipal": "Contb. Publica",
        "Contrib llumPublica Municipal": "Contb. Publica",
        "Contrib IlumPüblica Municipal": "Contb. Publica",
        "Contrib llum Pública Municipal": "Contb. Publica",
        "Contrib llum Püblca Municipal": "Contb. Publica",
        "Contrib llum Pública Municpal": "Contb. Publica",
        "Contrib llum Püblca Municipal": "Contb. Publica",
        "Contrib llum Pública Municpal": "Contb. Publica",
        "Contrib llum Püblica Municipal": "Contb. Publica",
        "Contrib llum Pública Municpal": "Contb. Publica",
        "Contrib Ilum Pública Municipal": "Contb. Publica",
        "Contrib Ilum Publica Municipal": "Contb. Publica",
        "Contrib IlumPublica Municipal": "Contb. Publica",
        "Contrib llumPublica Municipal": "Contb. Publica",
        "Contrib IlumPüblica Municipal": "Contb. Publica",
        "Contrib llum Pública Municipal": "Contb. Publica",
        "Contrib llum Püblca Municipal": "Contb. Publica",



#===================================================================Total===================================================================
        "TOTAL": "Total",
        "Total": "Total",
        "T0TAL": "Total",
        "TOTAl": "Total",
        "TQTAL": "Total"}


OUTPUT_DIR = "pages"
CROPPED_IMG_PATH = "ext_im.jpg"
EXCEL_PATH = "ocr_extracted_table.xlsx"
os.makedirs(OUTPUT_DIR, exist_ok=True)

senhas_conhecidas = ["33042", "56993", "60869", "08902", "3304", "5699", "6086", "0890"]

recortes_cemig = {
    "CMIN_ARCOS_DIST_EE": (50, 625, 1600, 1150),
    "CIBR_BARBACENA_DIST_EE": (50, 625, 1600, 1150),
    "CIBR_BARROSO_DIST_EE": (50, 625, 1600, 1150),
    "CMIN_CASA_DE_PEDRA_DIST_EE": (50, 625, 1600, 1150),
    "CIBR_PEDRO_LEOPOLDO_DIST_EE": (50, 625, 1600, 1150),
    "CMIN_PIRES_OURO_PRETO_DIST_EE": (50, 625, 1600, 1150),
    "INAL_UBERLANDIA_DIST_EE": (50, 625, 1600, 1150),
    "MIPE_ITABIRITO_DIST_EE": (50, 625, 1600, 1150),
    "CIBR_MONTES_CLAROS_DIST_EE": (50, 625, 1600, 1150),
    "CSN_IGARAPAVA_CONEXAO": (50, 625, 1600, 1150),
    "CIBR_PCH_MACACOS_CONEXAO": (50, 625, 1600, 1150),

}

recortes_light = {
    "CIBR_MARECHAL_DIST_EE": (50, 810, 1270, 1250),
    "CIBR_MARECHAL_DIST_ICMS": (50, 810, 1270, 1250),
    "CIBR_STA_CRUZ_RIO_BLENDER_DIST_EE": (50, 810, 1270, 1250),
    "CIBR_STA_CRUZ_RIO_BLENDER_DIST_ICMS": (50, 810, 1270, 1250),
    "CMIN_TECAR_TECON_DIST_EE": (50, 810, 1270, 1250),
    "CMIN_TECAR_TECON_DIST_ICMS": (50, 810, 1270, 1250),
    "CSN_CPES_DIST_EE": (50, 810, 1270, 1250),
    "CSN_DTIN_DIST_EE": (50, 810, 1270, 1250),
    "CSN_ESCORIA_DIST_ICMS": (50, 810, 1270, 1250),
    "CSN_ESCORIA_DIST_EE": (50, 810, 1270, 1250),
    "CSN_UPV_DIST_EE": (50, 810, 1270, 1250),
    "CSN_UPV_DIST_ICMS": (50, 810, 1270, 1250),
    "CMIN_TECAR_TECON_DIST_ICMS": (50,810,1270,1250)
}

recortes_ampla = {
    "CIBR_BARUERI_AGR_DIST_EE": (760, 830, 1600, 1150),
    "CIBR_CAJAMAR_AGR_DIST_EE": (760, 830, 1600, 1150),
    "CIBR_CANTAGALO_DIST_EE": (760, 830, 1600, 1150),
    "CIBR_CANTAGALO_DIST_ICMS": (760, 830, 1600, 1150),
    "CSN_PORTO_REAL_DIST_EE": (760, 830, 1600, 1150),
    "CSN_PORTO_REAL_DIST_ICMS": (760, 830, 1600, 1150),
    "INAL_RESENDE_DIST_EE": (760, 830, 1600, 1150),
    "INAL_RESENDE_DIST_ICMS": (760, 830, 1600, 1150),
    "INAL_SANTO_AMARO_DIST_EE": (760, 830, 1600, 1150),
    "CIBR_BARUERI_DIST_EE": (760, 830, 1600, 1150),
}


recortes_por_nome = {}
recortes_por_nome.update(recortes_cemig)
recortes_por_nome.update(recortes_light)
recortes_por_nome.update(recortes_ampla)

def desbloquear_pdf(caminho_pdf_original, caminho_pdf_desbloqueado, senhas):
    for senha in senhas:
        try:
            with pikepdf.open(caminho_pdf_original, password=senha) as pdf:
                pdf.save(caminho_pdf_desbloqueado)
            return True
        except pikepdf.PasswordError:
            continue
    return False

def contar_virgulas(txt):
    return txt.count(",")

def limpar_separadores_iniciais(txt):
    while txt and txt[0] in [".", ",", "0", " "]:
        txt = txt[1:]
    return txt

def split_valores_monetarios(txt):
    if txt.count(",") == 1 and "-" not in txt:
        return [txt.strip()]
    resultados = []
    i = 0
    temp = ""
    contador_virgulas = 0
    while i < len(txt):
        ch = txt[i]
        temp += ch
        if ch == ",":
            contador_virgulas += 1
            if contador_virgulas == 1 and i + 2 < len(txt):
                temp += txt[i + 1] + txt[i + 2]
                i += 2
                restante = txt[i + 1:]
                restante = limpar_separadores_iniciais(restante)
                if restante and (restante[0] == "-" or restante[0].isdigit()):
                    resultados.append(temp.strip())
                    outros = split_valores_monetarios(restante)
                    resultados.extend(outros)
                    return resultados
        i += 1
    return [txt.strip()] if not resultados else resultados

def corrigir_valores_complexos_monetarios(df):
    for i in range(1, len(df)):
        for j in range(len(df.columns)):
            celula = df.iat[i, j]
            if not isinstance(celula, str):
                continue
            celula = celula.strip()
            if celula and contar_virgulas(celula) >= 2:
                partes = split_valores_monetarios(celula)
                if len(partes) > 1:
                    for k in range(len(partes)):
                        if j + k < len(df.columns):
                            df.iat[i, j + k] = partes[k]
    return df

def formatar_valor_brasileiro(valor):
    try:
        if isinstance(valor, Decimal):
            valor_float = float(valor)
        else:
            valor_float = float(str(valor).replace(".", "").replace(",", "."))
        return f"{valor_float:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    except:
        return valor

def linha_so_icms(row):
    texto_completo = " ".join(str(cell).strip().upper() for cell in row if str(cell).strip())

    if "CONSUMO REATIVO EXCEDENTE NP" in texto_completo:
        return False
    palavras_descartaveis = {"ICMS"}
    if all(cell in palavras_descartaveis or "TRIBUTO" in cell for cell in texto_completo.split()):
        return True
    if all("ICMS" in cell for cell in texto_completo.split()):
        return True
    return False

def draw_boxes_and_extract_table(image_path, boxes, texts, horiz_lines, vert_lines, horiz_boxes, vert_boxes):
    image = cv2.imread(image_path)
    out_array = [["" for _ in range(len(vert_lines))] for _ in range(len(horiz_lines))]
    unordered_boxes = [vert_boxes[i][0] for i in vert_lines]
    ordered_boxes = np.argsort(unordered_boxes)
    def intersection(box_1, box_2):
        return [box_2[0], box_1[1], box_2[2], box_1[3]]
    def iou(box_1, box_2):
        x_1, y_1 = max(box_1[0], box_2[0]), max(box_1[1], box_2[1])
        x_2, y_2 = min(box_1[2], box_2[2]), min(box_1[3], box_2[3])
        inter = max(x_2 - x_1, 0) * max(y_2 - y_1, 0)
        if inter == 0:
            return 0
        area1 = (box_1[2] - box_1[0]) * (box_1[3] - box_1[1])
        area2 = (box_2[2] - box_2[0]) * (box_2[3] - box_2[1])
        return inter / float(area1 + area2 - inter)
    for i in range(len(horiz_lines)):
        for j in range(len(vert_lines)):
            res = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]])
            for b in range(len(boxes)):
                box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                if iou(res, box) > 0.1:
                    texto = texts[b].strip()
                    out_array[i][j] = texto
    df = pd.DataFrame(out_array)
    return image, df

def corrigir_cabecalho_cemig(df):
    df.columns = [
        "Itens Fatura",
        "Unidade",
        "Quantidade",
        "Preço Unitário",
        "Valor (R$)",
        "PIS/COFINS",
        "Base ICMS",
        "Aliquota ICMS",
        "ICMS (R$)",
        "Tarifa Unid.",
        "Extra"
    ][:len(df.columns)]
    return df


def corrigir_cabecalho_ampla(df):
    df.columns = [
        "Itens Fatura",
        "Unid.",
        "Quantidade",
        "Preço Unitário",
        "Valor (R$)",
        "PIS/COFINS",
        "ICMS (R$)",
        "Aliquota ICMS",
        "ICMS (R$)",
        "Tarifa Unit.",
        "Extra"
    ][:len(df.columns)]
    return df

def corrigir_cabecalho_light(df):
    df.columns = [
        "Itens Fatura",
        "Unid.",
        "Quantidade",
        "Preço Unitário",
        "Valor (R$)",
        "PIS/COFINS",
        "ICMS (R$)",
        "Aliquota ICMS",
        "ICMS (R$)",
        "Tarifa Unit.",
        "Extra"
    ][:len(df.columns)]
    return df

def corrigir_negativo_depois_ampla(df):
    def corrigir_valor(valor):
        if isinstance(valor, str):
            valor = valor.strip()
            if valor.endswith("-"):
                valor_corrigido = "-" + valor[:-1]
                return valor_corrigido
        return valor
    if "Valor (R$)" in df.columns:
        df["Valor (R$)"] = df["Valor (R$)"].apply(corrigir_valor)

    return df

def apagar_coluna_caso_extra_ampla(df):
    if "Extra" in df.columns:
        colunas = df.columns.tolist()
        idx_extra = colunas.index("Extra")

        for i in range(len(df)):
            for j in range(1, idx_extra + 1):
                df.iat[i, j - 1] = df.iat[i, j]
            df.iat[i, idx_extra] = ""
        df.columns = [
            "Itens Fatura", "Unid.", "Quantidade", "Preço Unitário",
            "Valor (R$)", "PIS/COFINS", "ICMS (R$)", "Aliquota ICMS",
            "ICMS (R$)", "Tarifa Unit.", "Extra"
        ][:len(df.columns)]

    return df

def corrigir_valores_negativos_gerais_ampla(df):
    colunas_valores = ["Valor (R$)", "PIS/COFINS", "ICMS (R$)"]
    palavras_chave_negativas = ["DEDUCAO ENERGIA", "DESCONTO COMP", "AJUSTE DE DESCONTO", "BENEFICIO TARIFARIO LIQUIDO","TAR.REDUZ.RES.ANEEL166/05FP"]

    for idx, row in df.iterrows():
        descricao = str(row["Itens Fatura"]).upper()
        if any(palavra in descricao for palavra in palavras_chave_negativas):
            for col in colunas_valores:
                try:
                    valor = str(df.at[idx, col]).replace(".", "").replace(",", ".")
                    valor_float = float(valor)
                    if valor_float > 0:
                        df.at[idx, col] = f"-{valor_float:.2f}".replace(".", ",")
                except:
                    continue
    return df



def normalizar_texto(texto):
    if not isinstance(texto, str):
        texto = str(texto)
    texto = texto.upper()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    texto = re.sub(r'[^A-Z0-9 ]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def encontrar_tipo_aproximado(texto_item):
    texto_item_norm = normalizar_texto(texto_item)
    for chave, tipo in mapeamento_tipo_cobranca.items():
        chave_norm = normalizar_texto(chave)
        if texto_item_norm == chave_norm:
            return tipo
    return None

def corrigir_valores_absurdos_com_negativo(df, debug=False):
    for i in df.index:
        try:
            valor_str = str(df.at[i, "Valor (R$)"]).replace(".", "").replace(",", ".")
            valor_float = float(valor_str)


            if valor_float < -1000000:
                valor_corrigido = valor_float / 100
                valor_formatado = f"{valor_corrigido:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
                df.at[i, "Valor (R$)"] = valor_formatado
                if debug:
                    print(f"[Linha {i}] Corrigido OCR: {valor_float} → {valor_corrigido}")
        except Exception as e:
            if debug:
                print(f"[Linha {i}] Erro: {e}")
            continue
    return df



def classificar_tipo_ampla_desc_encargo(df):
    def normalizar(txt):
        if not isinstance(txt, str):
            txt = str(txt)
        txt = unicodedata.normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
        txt = re.sub(r'[^A-Z0-9 ]', '', txt.upper())
        return txt
    def detectar_tipo(texto_item):
        texto_item_norm = normalizar(texto_item)
        if "DEDUCAO" in texto_item_norm and "ENERGIA" in texto_item_norm:
            return "Desc. Encargo"
        for chave, tipo in mapeamento_tipo_cobranca.items():
            if normalizar(chave) in texto_item_norm:
                return tipo
        return "Outros"
    df["Tipo de Cobrança"] = df["Itens Fatura"].apply(detectar_tipo)
    return df


def remover_subtotais(df):
    padroes_excluir = [
        "subtotal faturamento",
        "subtotal outros"
    ]

    def is_subtotal(texto):
        if not isinstance(texto, str):
            return False
        texto_normalizado = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII').lower().strip()
        return any(p in texto_normalizado for p in padroes_excluir)

    return df[~df["Itens Fatura"].apply(is_subtotal)].reset_index(drop=True)


def manter_linha_consumo_reativo(df):
    padrao_base = "consumo reativo excedente"

    def normalizar_texto(txt):
        if not isinstance(txt, str):
            txt = str(txt)
        txt = unicodedata.normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
        txt = re.sub(r'[^a-zA-Z0-9 ]', '', txt)
        return txt.lower().strip()

    def manter_linha(row):
        texto = normalizar_texto(row.get("Itens Fatura", ""))
        return padrao_base in texto or texto != ""

    return df[df.apply(manter_linha, axis=1)].reset_index(drop=True)
#====================================================================CEMIG=======================================================================================
def processar_fatura_cemig(df):
    df = df[~df.apply(linha_so_icms, axis=1)].reset_index(drop=True)
    df = df[~df.iloc[:, 0].astype(str).str.strip().str.lower().str.contains(r"^itens\s+.*fatura")].reset_index(drop=True)
    if df.shape[0] > 1 and df.iloc[1].astype(str).str.contains(r"unit.*r\$|valor|cofins|icms", case=False).sum() >= 2:
        df = df.drop(index=1).reset_index(drop=True)
    df = corrigir_cabecalho_cemig(df)
    df = corrigir_valores_complexos_monetarios(df)

    for i in range(1, len(df)):
        for j in range(len(df.columns)):
            df.iat[i, j] = formatar_valor_brasileiro(df.iat[i, j])
    def normalizar_texto(txt):
        txt = unicodedata.normalize('NFKD', str(txt)).encode('ASCII', 'ignore').decode('ASCII')
        txt = re.sub(r'\s+', ' ', txt).replace(".", "").replace(",", "").lower().strip()
        return txt

    def encontrar_tipo_aproximado(texto_item):
        texto_item_norm = normalizar_texto(texto_item)
        for chave, tipo in mapeamento_tipo_cobranca.items():
            if normalizar_texto(chave) in texto_item_norm:
                return tipo
        return None
    df_tipo = df.copy()
    df_tipo["Valor_float"] = df_tipo["Valor (R$)"].apply(lambda x: str(x).replace(".", "").replace(",", "."))
    df_tipo["Valor_float"] = pd.to_numeric(df_tipo["Valor_float"], errors="coerce")
    df_tipo["Tipo de Cobrança"] = df_tipo["Itens Fatura"].apply(
        lambda x: encontrar_tipo_aproximado(x) or "Outros"
    )
    resumo = df_tipo.groupby("Tipo de Cobrança", dropna=True)["Valor_float"].sum().reset_index()
    resumo.columns = ["Tipo de Cobrança", "Valor Total (R$)"]
    resumo["Valor Total (R$)"] = resumo["Valor Total (R$)"].apply(
        lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    )
    return df, resumo




#====================================================================LIGHT=======================================================================================
def processar_fatura_light(df):
    df = df[~df.apply(linha_so_icms, axis=1)].reset_index(drop=True)
    df = df[~df.iloc[:, 0].astype(str).str.strip().str.lower().str.contains(r"^itens\s+.*fatura")].reset_index(drop=True)

    if df.shape[0] > 1 and df.iloc[1].astype(str).str.contains(r"unit.*r\$|valor|cofins|icms", case=False).sum() >= 2:
        df = df.drop(index=1).reset_index(drop=True)

    df = df.loc[:, ~df.columns.duplicated()]
    df = corrigir_cabecalho_light(df)
    df = corrigir_valores_complexos_monetarios(df)

    for i in range(1, len(df)):
        for j in range(len(df.columns)):
            df.iat[i, j] = formatar_valor_brasileiro(df.iat[i, j])

    df = df[df["Itens Fatura"].astype(str).str.strip() != ''].reset_index(drop=True)

    def normalizar_texto(txt):
        if not isinstance(txt, str):
            txt = str(txt)
        txt = unicodedata.normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
        txt = re.sub(r'\s+', ' ', txt).replace(".", "").replace(",", "").lower().strip()
        return txt

    def encontrar_tipo_aproximado(texto_item):
        texto_item_norm = normalizar_texto(texto_item)
        for chave, tipo in mapeamento_tipo_cobranca.items():
            if normalizar_texto(chave) in texto_item_norm:
                return tipo
        return None

    df_tipo = df.copy()
    df_tipo["Valor_float"] = df_tipo["Valor (R$)"].apply(lambda x: str(x).replace(".", "").replace(",", "."))
    df_tipo["Valor_float"] = pd.to_numeric(df_tipo["Valor_float"], errors="coerce")
    df_tipo["Tipo de Cobrança"] = df_tipo["Itens Fatura"].apply(
        lambda x: encontrar_tipo_aproximado(x) or "Outros"
    )

    resumo = df_tipo.groupby("Tipo de Cobrança", dropna=True)["Valor_float"].sum().reset_index()
    resumo.columns = ["Tipo de Cobrança", "Valor Total (R$)"]
    resumo["Valor Total (R$)"] = resumo["Valor Total (R$)"].apply(
        lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    )

    return df, resumo



#====================================================================AMPLA=======================================================================================
def processar_fatura_ampla(df):
    df = df[~df.apply(lambda row: row.astype(str).str.lower().str.contains("itens.*fatura").any(), axis=1)].reset_index(drop=True)
    if df.shape[0] > 1 and df.iloc[1].astype(str).str.contains(r"unit.*r\$|valor|cofins|icms", case=False).sum() >= 2:
        df = df.drop(index=1).reset_index(drop=True)
    df = corrigir_cabecalho_ampla(df)
    df = apagar_coluna_caso_extra_ampla(df)
    df = corrigir_valores_negativos_gerais_ampla(df)
    df = corrigir_negativo_depois_ampla(df)
    df = df[~df.apply(linha_so_icms, axis=1)].reset_index(drop=True)
    df = corrigir_valores_complexos_monetarios(df)
    df = corrigir_valores_absurdos_com_negativo(df)
    df = classificar_tipo_ampla_desc_encargo(df)
    df = manter_linha_consumo_reativo(df)
    for i in range(1, len(df)):
        for j in range(len(df.columns)):
            df.iat[i, j] = formatar_valor_brasileiro(df.iat[i, j])
    df = df[df["Itens Fatura"].astype(str).str.strip() != ''].reset_index(drop=True)
    df_tipo = df.copy()
    df_tipo["Valor_float"] = df_tipo["Valor (R$)"].apply(lambda x: str(x).replace(".", "").replace(",", "."))
    df_tipo["Valor_float"] = pd.to_numeric(df_tipo["Valor_float"], errors="coerce")
    df_tipo = remover_subtotais(df_tipo)
    if "Tipo de Cobrança" not in df_tipo.columns:
        df_tipo = classificar_tipo_ampla_desc_encargo(df_tipo)
    resumo = df_tipo.groupby("Tipo de Cobrança", dropna=True)["Valor_float"].sum().reset_index()
    resumo.columns = ["Tipo de Cobrança", "Valor Total (R$)"]
    resumo["Valor Total (R$)"] = resumo["Valor Total (R$)"].apply(
        lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    )
    return df, resumo

In [17]:
# @title Leitor de OCR
# @markdown <b>Esta célula faz a leitura com DeepLearning e Classifica em Caixar por Intervalo de Confiança, considerando problemas específicos para cada distribuidora.</b>

from IPython.display import display
import ipywidgets as widgets

# Botão de upload mais chamativo
upload_button = widgets.FileUpload(
    accept='.pdf',
    multiple=True,
    description='📤 ENVIAR PDF(S)',
    style={
        'description_width': 'initial',
        'button_color': '#1B9C00',  # Azul escuro
        'font_weight': 'bold',
    },
    layout=widgets.Layout(
        width='300px',
        height='45px',
        border='2px solid #0077FF',
        margin='10px 0px 10px 0px',
        font_size='15px',
    )
)

status = widgets.Output()
display(widgets.VBox([upload_button, status]))

def processar_pdfs(change):
    if not upload_button.value:
        with status:
            print("⚠️ Nenhum arquivo enviado.")
        return

    arquivos = upload_button.value
    clear_output()
    display(widgets.VBox([upload_button, status]))
    arquivos_validos = 0
    mapa_processamento = {
        (50, 625, 1600, 1150): processar_fatura_cemig,
        (50, 810, 1270, 1250): processar_fatura_light,
        (760, 830, 1600, 1150): processar_fatura_ampla,
    }

    with status:
        print(f"🕖️ Processando {len(arquivos)} arquivo(s)...\n")

    with pd.ExcelWriter(EXCEL_PATH, engine='openpyxl', mode='w') as writer:
        for nome_pdf in arquivos:
            with status:
                print(f"🔍 Extraindo dados de: {nome_pdf}")

            conteudo = arquivos[nome_pdf]['content']
            with open(nome_pdf, 'wb') as f:
                f.write(conteudo)

            nome_limpo = os.path.splitext(nome_pdf)[0]
            partes = nome_limpo.split("_", 1)
            if len(partes) < 2:
                with status:
                    print(f"⚠️ Nome inesperado: {nome_pdf}. Pulando.")
                continue
            nome_base = partes[1]
            if nome_base not in recortes_por_nome:
                with status:
                    print(f"⛔️ Nome não encontrado na tabela de recortes: {nome_base}. Pulando arquivo.")
                continue

            arquivos_validos += 1
            X1, Y1, X2, Y2 = recortes_por_nome[nome_base]
            func = mapa_processamento.get((X1, Y1, X2, Y2))
            if func is None:
                with status:
                    print(f"⚠️ Recorte não associado a função: {nome_base}")
                continue

            pdf_desbloqueado_path = f"desbloqueado_{nome_pdf}"
            try:
                with pikepdf.open(nome_pdf):
                    shutil.copy(nome_pdf, pdf_desbloqueado_path)
            except pikepdf.PasswordError:
                if not desbloquear_pdf(nome_pdf, pdf_desbloqueado_path, senhas_conhecidas):
                    with status:
                        print(f"⛔️ Pulando arquivo protegido sem senha conhecida: {nome_pdf}")
                    continue

            images = convert_from_path(pdf_desbloqueado_path)
            page0_path = os.path.join(OUTPUT_DIR, "page0.jpg")
            images[0].save(page0_path, "JPEG")

            image = cv2.imread(page0_path)
            cropped = image[Y1:Y2, X1:X2]
            cv2.imwrite(CROPPED_IMG_PATH, cropped)

            output = ocr.ocr(CROPPED_IMG_PATH)
            boxes, texts, probs = [], [], []
            for line in output[0]:
                box, (text, prob) = line[0], line[1]
                boxes.append(box)
                texts.append(text)
                probs.append(prob)

            image_height, image_width = cropped.shape[:2]
            horiz_boxes, vert_boxes = [], []
            adjusted_horiz_boxes, adjusted_vert_boxes = [], []

            for box in boxes:
                x_h, x_v = 0, int(box[0][0])
                y_h, y_v = int(box[0][1]), 0
                width_h, width_v = image_width, int(box[2][0] - box[0][0])
                height_h, height_v = int(box[2][1] - box[0][1]), image_height

                horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
                vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])
                adjusted_horiz_boxes.append([y_h, x_h, y_h + height_h, x_h + width_h])
                adjusted_vert_boxes.append([y_v, x_v, y_v + height_v, x_v + width_v])

            adjusted_horiz_boxes = np.array(adjusted_horiz_boxes, dtype=np.float32)
            adjusted_vert_boxes = np.array(adjusted_vert_boxes, dtype=np.float32)
            probs = np.array(probs, dtype=np.float32)
            nms_threshold = 0.1

            horiz_out = tf.image.non_max_suppression(adjusted_horiz_boxes, probs, 1000, nms_threshold)
            vert_out = tf.image.non_max_suppression(adjusted_vert_boxes, probs, 1000, nms_threshold)
            horiz_lines = np.sort(horiz_out.numpy())
            vert_lines = np.sort(vert_out.numpy())

            _, extracted_df = draw_boxes_and_extract_table(
                CROPPED_IMG_PATH, boxes, texts,
                horiz_lines, vert_lines, horiz_boxes, vert_boxes
            )
            resultado = func(extracted_df)
            if isinstance(resultado, tuple):
                extracted_df, resumo = resultado
            else:
                resumo = None

            aba = nome_limpo[:31]
            extracted_df.to_excel(writer, sheet_name=aba, index=False)
            worksheet = writer.sheets[aba]

            for column_cells in worksheet.columns:
                max_length = max(len(str(cell.value)) if cell.value else 0 for cell in column_cells)
                column_letter = column_cells[0].column_letter
                worksheet.column_dimensions[column_letter].width = max_length + 2

            if resumo is not None:
                ordem_personalizada = [
                    "Encargo", "Reativa", "Desc. Encargo", "Demanda", "Desc. Demanda", "Contb. Pública", "Total"
                ]
                resumo["ordem"] = resumo["Tipo de Cobrança"].apply(
                    lambda x: ordem_personalizada.index(x) if x in ordem_personalizada else len(ordem_personalizada)
                )
                resumo = resumo.sort_values("ordem").drop(columns="ordem")

                bold_font = Font(bold=True)
                worksheet.cell(row=2, column=13, value="Tipo de Cobrança").font = bold_font
                worksheet.cell(row=2, column=14, value="Valor Total (R$)").font = bold_font

                for r_idx, row in enumerate(resumo.itertuples(index=False), start=3):
                    worksheet.cell(row=r_idx, column=13, value=row[0])
                    worksheet.cell(row=r_idx, column=14, value=row[1])

                for col_idx in [13, 14]:
                    col = worksheet.cell(row=2, column=col_idx).column_letter
                    max_length = max(
                        len(str(worksheet.cell(row=r, column=col_idx).value)) for r in range(2, 3 + len(resumo))
                    )
                    worksheet.column_dimensions[col].width = max_length + 2

            try:
                os.remove(pdf_desbloqueado_path)
                os.remove(nome_pdf)
            except Exception as e:
                with status:
                    print(f"⚠️ Erro ao deletar arquivos temporários: {e}")

    if arquivos_validos == 0:
        with status:
            print("\n⚠️ SEM ARQUIVOS VÁLIDOS PARA LEITURA")
    else:
        with status:
            print(f"\n✅ {arquivos_validos} arquivos processados com sucesso. Arquivo salvo: {EXCEL_PATH}")
            files.download(EXCEL_PATH)

upload_button.observe(processar_pdfs, names='value')
